<h1>Scrape e-commerce data search</h1>
<br>
<h4>by : @qywok_exploiter</h4>
<hr>
<h2>e-commerce</h2>
<h3> -> Tokopedia</h3>
<h3> -> Shopee</h3>
<h3> -> Bukalapak</h3>
<br>

In [1]:
#BERCANDA
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://media.tenor.com/images/33c8c26810ae39857de47b663463cd66/tenor.gif")

In [2]:
import re
import json
import urllib
import pandas as pd
import cloudscraper
import requests as req
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

<br>

**[ 1 ] TOKOPEDIA**

In [3]:
class Tokopedia:
    
    def __init__(self, Search = "ban bekas") :
        
        self.PARAMS   = f'device=mobile&ob=23&page=1&q={Search}&related=true&rows=999999999&safe_search=false&scheme=https&shipping=&source=search&st=product&start=0&topads_bucket=true&unique_id=504d5a929adeabc5d177d7473394f0f7&user_addressId=&user_cityId=&user_districtId=&user_id=&user_lat=&user_long=&user_postCode=&variants='
        self.ENDPOINT = "https://gql.tokopedia.com/"
        self.PAYLOAD  =  {
            
            'operationName': 'SearchProductQueryV4',
            
            'variables': 
                {'params': 
                 
                     self.PARAMS
                 
                },
            
             'query': 'query SearchProductQueryV4($params: String!) {\n  ace_search_product_v4(params: $params) {\n    header {\n      totalData\n      totalDataText\n      processTime\n      responseCode\n      errorMessage\n      additionalParams\n      keywordProcess\n      __typename\n    }\n    data {\n      isQuerySafe\n      ticker {\n        text\n        query\n        typeId\n        __typename\n      }\n      redirection {\n        redirectUrl\n        departmentId\n        __typename\n      }\n      related {\n        relatedKeyword\n        otherRelated {\n          keyword\n          url\n          product {\n            id\n            name\n            price\n            imageUrl\n            rating\n            countReview\n            url\n            priceStr\n            wishlist\n            shop {\n              city\n              isOfficial\n              isPowerBadge\n              __typename\n            }\n            ads {\n              adsId: id\n              productClickUrl\n              productWishlistUrl\n              shopClickUrl\n              productViewUrl\n              __typename\n            }\n            badges {\n              title\n              imageUrl\n              show\n              __typename\n            }\n            ratingAverage\n            labelGroups {\n              position\n              type\n              title\n              __typename\n            }\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      suggestion {\n        currentKeyword\n        suggestion\n        suggestionCount\n        instead\n        insteadCount\n        query\n        text\n        __typename\n      }\n      products {\n        id\n        name\n        ads {\n          adsId: id\n          productClickUrl\n          productWishlistUrl\n          productViewUrl\n          __typename\n        }\n        badges {\n          title\n          imageUrl\n          show\n          __typename\n        }\n        category: departmentId\n        categoryBreadcrumb\n        categoryId\n        categoryName\n        countReview\n        discountPercentage\n        gaKey\n        imageUrl\n        labelGroups {\n          position\n          title\n          type\n          __typename\n        }\n        originalPrice\n        price\n        priceRange\n        rating\n        ratingAverage\n        shop {\n          id\n          name\n          url\n          city\n          isOfficial\n          isPowerBadge\n          __typename\n        }\n        url\n        wishlist\n        sourceEngine: source_engine\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n'}
        
    def getTokopedia(self):
        
        sorting          = []
        request_products = json.loads(req.post(self.ENDPOINT, json = self.PAYLOAD).content)
        
        def price(value):
            
            Price = ""
            
            for indexPrice in range(len(value)):
                
                Index = format(ord(value[indexPrice]), "x")
                if (Index == "52")|(Index == "70")|(Index == "2e") :
                    
                    pass
                
                else:
                    Price+=value[indexPrice]
            
            
            return int(Price)
            
        
        for indexProducts in range(len(request_products["data"]["ace_search_product_v4"]["data"]["products"])) :
            
            Product = {
                
                "nama_produk" : request_products["data"]["ace_search_product_v4"]["data"]["products"][indexProducts]["name"],
                "harga"       : price(request_products["data"]["ace_search_product_v4"]["data"]["products"][indexProducts]["price"]),
                "diskon"      : request_products["data"]["ace_search_product_v4"]["data"]["products"][indexProducts]["discountPercentage"],
                "rating"      : float(request_products["data"]["ace_search_product_v4"]["data"]["products"][indexProducts]["ratingAverage"]),
                "foto_produk" : request_products["data"]["ace_search_product_v4"]["data"]["products"][indexProducts]["imageUrl"],
                "url_produk"  : request_products["data"]["ace_search_product_v4"]["data"]["products"][indexProducts]["url"],
                "nama_toko"   : request_products["data"]["ace_search_product_v4"]["data"]["products"][indexProducts]["shop"]["name"],
                "lokasi_toko" : request_products["data"]["ace_search_product_v4"]["data"]["products"][indexProducts]["shop"]["city"],
                "url_toko"    : request_products["data"]["ace_search_product_v4"]["data"]["products"][indexProducts]["shop"]["url"],
                "asal_ecommerce" : "tokopedia"
                
                
            }
            
            sorting.append(
            
                Product
            
            )
            
#         return pd.DataFrame(sorting).sort_values(by=["harga","rating","diskon"])
        return sorting
            
    

<br>

**[ 2 ] SHOPEE**

In [4]:
class Shopee:
    
    def __init__(self, Search = "ban"):
        
        self.ENDPOINT = f"https://shopee.co.id/api/v4/search/search_items?by=relevancy&keyword={Search}&limit=100&newest=100&order=desc&page_type=search&scenario=PAGE_GLOBAL_SEARCH&version=2"
        self.IMAGE_ENDPOINT = "https://cf.shopee.co.id/file/"
        self.SHOP = "https://shopee.co.id/api/v4/product/get_shop_info?shopid="
        
        
    def getShopee(self) :
        
        sorting          = []
        request_products = json.loads(req.get(self.ENDPOINT).content)
        
        for indexProducts in range(len(request_products["items"])) :
            
            price        = str(request_products["items"][indexProducts]["item_basic"]["price"])
            discount     = str(request_products["items"][indexProducts]["item_basic"]["discount"])
            rating       = str(request_products["items"][indexProducts]["item_basic"]["item_rating"]["rating_star"])
            requestShop  = json.loads(req.get("https://shopee.co.id/api/v4/product/get_shop_info?shopid="+str(request_products["items"][indexProducts]["item_basic"]["shopid"])).content)
            
            def urlProduk():
                
                endpoint = "https://shopee.co.id/-"
                Name     = request_products["items"][indexProducts]["item_basic"]["name"]
                result   = ""
                shopid   = str(request_products["items"][indexProducts]["item_basic"]["shopid"])
                itemid   = str(request_products["items"][indexProducts]["item_basic"]["itemid"])
                for indexResult in range(len(Name)):
                    if format(ord(Name[indexResult]), "x") == "20":
                        result+="-"
                    else:
                        result+=Name[indexResult]
                return f"{endpoint}{result}-i.{shopid}.{itemid}"
            
            
            Product = {
                
                "nama_produk" : request_products["items"][indexProducts]["item_basic"]["name"],
                "harga"       : int(price[0:len(price)-5]),
                "diskon"      : int(discount[0:len(discount)-1]) if type(discount[0:len(discount)-1]) == "int" else discount[0:len(discount)-1]  ,
                "rating"      : float(rating[0:3]),
                "foto_produk" : self.IMAGE_ENDPOINT+request_products["items"][indexProducts]["item_basic"]["image"],
                "url_produk"  : urlProduk(),
                "nama_toko"   : requestShop["data"]["account"]["username"],
                "lokasi_toko" : requestShop["data"]["shop_location"],
                "url_toko"    : "https://shopee.co.id/"+requestShop["data"]["account"]["username"],
                "asal_ecommerce" : "shopee"
                
                
            }
            
            sorting.append(
            
                Product
            
            )
            
        return sorting

        

<br>

**[ 3 ] Bukalapak**

In [5]:
class Bukalapak:
    
    def __init__(self, Search):
        
        self.ENDPOINT         = f"https://api.bukalapak.com/multistrategy-products?prambanan_override=true&keywords={Search}&limit=50&offset=50&facet=true&page=1&access_token="
        self.TOKEN_ENDPOINT  = f"http://www.bukalapak.com/404?marketplace_path=%2Fu%2Fwesteros_auth_proxies"
        
        self.cookies_token = {
                
                "__asc" : "a3e0eb9e17a03a6e7a5988fed4f",
                "__auc": "19928746177874debc7bca2450d",
                "__cfruid" : "9d80714daf3fb1ed5e80d7e10123c28d8ec630cc-1623556183",
                "_dd_s":"logs=1&id=c00b0782-a851-4b87-84b3-6167846a3936&created=1623556188705&expire=1623560339953",
                "_fbp":"fb.1.1623524287461.1658652966",
                "_ga":"GA1.2.864919285.1623524285",
                "_gcl_au":"1.1.934570982.1623524285",
                "_gid":"GA1.2.293862253.1623524285",
                "_hjAbsoluteSessionInProgress":"1",
                "_hjid":"ee5bbddd-8300-48a8-9a0b-b3d71f96a07c",
                "_hjIncludedInSessionSample":"1",
                "_hjTLDTest":"1",
                "_td":"477254b3-df17-44f4-adc6-feb14c354d9c",
                "_vwo_uuid_v2":"DB279B8E19F57F720EEA02225A0A044EF|32ed062cbc20a26c644b5144a3694c5f",
                "ab.storage.deviceId.a987eba1-624b-42ab-ba73-75c895c038fd":'{"g":"7ece91da-843d-7367-0e53-5b0fb8674a02","c":1623524294771,"l":1623524294771}',
                "browser_id":"5f9c869cb72a70c17301aec27d5d1e09",
                "cto_bundle":"2k9gpF92RUlOMHdJaGRVU1ElMkZrRWx0JTJCTXdleG5HM1p5eTA3bHk0WEclMkJwc1lZOWg3dWdjMjJJdW9SUWtkSklnVm9HYXZrSWY2NiUyQkUwaXRiUU5PeGs5d25mJTJCaCUyQm5hdnFsTER6dk5XdDhVZDclMkZDJTJCQlB5Qjd0T053OCUyQkdnJTJCencxTjNJdWxEbnBJeE9oU0RkanNiMzJRbEslMkYlMkJ2M2clM0QlM0Q",
                "identity":"f605042192d5be2b30c78e6b4322cb63",
                "kppid_managed":"kppidff_OLWOYDUw",
                "lskjfewjrh34ghj23brjh234":"a0dqRGZUSS96ZnF1MVRkeGRqVGloaHovZ3o4YzFXT3ZUS3lsM0dWUFJjS0hKNytRdXRmYlV3V2dHNXVDdTJIVGNTZnM1R2Qxb2JYV2U3YTBWM01zd2xiQjFLM3RTUmIrcDdDOTQvMENnaEhJcU5JcDJxdGpKTytOWkRpTjdFY0l4WkloMXhvUzd3cFFsY1NlSkQrMDhzQUkxd2tQM0FRSHpmRFFqcVB6OE5FQzZ5clMzbk9qRmNjRFRiME8vaWlrLS1xV0k3NGFEaEhERlhHWnYyQkpyM3hBPT0=--e6c0ff3abeddb96ccbc4ded5549b9b2eaff204cd",
                "session_id":""
                
            }
        
        self.headers_token = {
                
                "Host": "www.bukalapak.com",
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
                "Accept": "application/json, text/plain, */*",
                "Accept-Language": "en-US,en;q=0.5",
                "Accept-Encoding": "gzip, deflate, br",
                "Content-Type": "application/json;charset=utf-8",
                "Content-Length": "44",
                "Origin": "https://www.bukalapak.com",
                "Referer": "https://www.bukalapak.com/",
                "Connection": "keep-alive",
                "Cookie": "identity=f605042192d5be2b30c78e6b4322cb63; browser_id=5f9c869cb72a70c17301aec27d5d1e09; __auc=19928746177874debc7bca2450d; session_id=; lskjfewjrh34ghj23brjh234=a0dqRGZUSS96ZnF1MVRkeGRqVGloaHovZ3o4YzFXT3ZUS3lsM0dWUFJjS0hKNytRdXRmYlV3V2dHNXVDdTJIVGNTZnM1R2Qxb2JYV2U3YTBWM01zd2xiQjFLM3RTUmIrcDdDOTQvMENnaEhJcU5JcDJxdGpKTytOWkRpTjdFY0l4WkloMXhvUzd3cFFsY1NlSkQrMDhzQUkxd2tQM0FRSHpmRFFqcVB6OE5FQzZ5clMzbk9qRmNjRFRiME8vaWlrLS1xV0k3NGFEaEhERlhHWnYyQkpyM3hBPT0%3D--e6c0ff3abeddb96ccbc4ded5549b9b2eaff204cd; __cfruid=9d80714daf3fb1ed5e80d7e10123c28d8ec630cc-1623556183; _ga=GA1.2.864919285.1623524285; _gid=GA1.2.293862253.1623524285; _gcl_au=1.1.934570982.1623524285; _fbp=fb.1.1623524287461.1658652966; kppid_managed=kppidff_OLWOYDUw; _hjTLDTest=1; _hjid=ee5bbddd-8300-48a8-9a0b-b3d71f96a07c; cto_bundle=2k9gpF92RUlOMHdJaGRVU1ElMkZrRWx0JTJCTXdleG5HM1p5eTA3bHk0WEclMkJwc1lZOWg3dWdjMjJJdW9SUWtkSklnVm9HYXZrSWY2NiUyQkUwaXRiUU5PeGs5d25mJTJCaCUyQm5hdnFsTER6dk5XdDhVZDclMkZDJTJCQlB5Qjd0T053OCUyQkdnJTJCencxTjNJdWxEbnBJeE9oU0RkanNiMzJRbEslMkYlMkJ2M2clM0QlM0Q; ab.storage.deviceId.a987eba1-624b-42ab-ba73-75c895c038fd=%7B%22g%22%3A%227ece91da-843d-7367-0e53-5b0fb8674a02%22%2C%22c%22%3A1623524294771%2C%22l%22%3A1623524294771%7D; _dd_s=logs=1&id=c00b0782-a851-4b87-84b3-6167846a3936&created=1623556188705&expire=1623560339953; _hjIncludedInSessionSample=1; _hjAbsoluteSessionInProgress=1; _vwo_uuid_v2=DB279B8E19F57F720EEA02225A0A044EF|32ed062cbc20a26c644b5144a3694c5f; _td=477254b3-df17-44f4-adc6-feb14c354d9c; __asc=a3e0eb9e17a03a6e7a5988fed4f",
                "Cache-Control": "max-age=0",
                "TE": "Trailers"
                
            }
        
        self.headers_endpoint = {
                
                "Host": "api.bukalapak.com",
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
                "Accept": "application/json, text/plain, */*",
                "Accept-Language": "en-US,en;q=0.5",
                "Accept-Encoding": "gzip, deflate, br",
                "Origin": "https://www.bukalapak.com",
                "Referer": "https://www.bukalapak.com/",
                "Connection": "keep-alive",
                "TE": "Trailers"
                
            }
        
        self.cookies_endpoint = {
                
                "__asc" : "b558002717a0436b60305005513",
                "__auc": "19928746177874debc7bca2450d",
                "_fbp":"fb.1.1623572747148.1262560140",
                "_gat_UA-12425854-1":"1",
                "_gat":"1",
                "_ga":"GA1.2.864919285.1623524285",
                "_gcl_au":"1.1.934570982.1623524285",
                "_gid":"GA1.2.293862253.1623524285",
                "_hjAbsoluteSessionInProgress":"0",
                "_hjFirstSeen":"1",
                "_hjid":"ee5bbddd-8300-48a8-9a0b-b3d71f96a07c",
                "_hjTLDTest":"1",
                "_td":"477254b3-df17-44f4-adc6-feb14c354d9c",
                "_vwo_uuid_v2":"DB279B8E19F57F720EEA02225A0A044EF|32ed062cbc20a26c644b5144a3694c5f",
                "ab.storage.deviceId.a987eba1-624b-42ab-ba73-75c895c038fd":'{"g":"7ece91da-843d-7367-0e53-5b0fb8674a02","c":1623524294771,"l":1623524294771}',
                "browser_id":"5f9c869cb72a70c17301aec27d5d1e09",
                "cto_bundle":"8Ogo9F92RUlOMHdJaGRVU1ElMkZrRWx0JTJCTXdlNGg3S3ZGNVQxdmNIVmFXYVd2MDNWeXgya003bTlzQnBNYURLY0tUenNjY2Q4b0J5MGx3Qzk3TzdRTHI1bnl5ZHp0bTRFOHJNaUNxUndyZWVFMiUyQmZWZUpkUUhZeUl0c3Q0VWNyN3pCV2todzFRYnpTRnZQbExscXp1MVhsMldsOGclM0QlM0Q",
                "identity":"f605042192d5be2b30c78e6b4322cb63",
                "lskjfewjrh34ghj23brjh234":"NmQzNGhpZmJ4MzJCZisxVFZXMjlNd1JHL00wSmViTWlZQ2lGdzcxa0RGZkdGdmdIS0V1MmwyRGt4UTJwbXFMNk1wMDFFUUhzS0pneUV5blBrcHBza04rOTZGajdMT1JSOFM0djRjb09YeW5CYUpVZGliQzkrR3BOVlFuZ2QrbmJnNXRiVW9DZlhJa25scFJKaG1YL0NhUDNjY0tZTGZBdDY4M0xNaXdKVTJGdVBPVEo3QkpQa0NGZmxyTXVvdjNHLS1yT1JTUFFHWTJSV1lDTEJWaTA3S2hRPT0%3D--9c05ee25869348a89781e38c2d8950b28da9bd3a",
                "session_id":""
                
            }
        
    def getBukalapak(self) :
        
        def findAccess_token():
            
            request = req.get(self.TOKEN_ENDPOINT, headers=self.headers_token, cookies=self.cookies_token)
            parseRequest = BeautifulSoup(request.text, 'html.parser')
            scripts = BeautifulSoup(str(parseRequest.find('meta',attrs={'name':'oauth-access-token'})), 'html.parser').find("meta")
            
            return str(scripts)[15:61]
            
        
        sorting = []
        scraper = cloudscraper.create_scraper()
        scraper = cloudscraper.CloudScraper()
        request_product = json.loads(scraper.get(self.ENDPOINT+findAccess_token()).text)
        
        for indexSorting in range(len(request_product["data"])):
            
            Product = {
                
                "nama_produk" : request_product["data"][indexSorting]["name"],
                "harga"       : int(request_product["data"][indexSorting]["price"]),
                "diskon"      : int(request_product["data"][indexSorting]["discount_percentage"]),
                "rating"      : float(request_product["data"][indexSorting]["rating"]["average_rate"]),
                "foto_produk" : request_product["data"][indexSorting]["images"]["large_urls"][0],
                "url_produk"  : request_product["data"][indexSorting]["url"],
                "nama_toko"   : request_product["data"][indexSorting]["store"]["name"],
                "lokasi_toko" : f'{request_product["data"][indexSorting]["store"]["address"]["city"]} - {request_product["data"][indexSorting]["store"]["address"]["province"]}',
                "url_toko"    : request_product["data"][indexSorting]["store"]["url"],
                "asal_ecommerce" : "bukalapak"
                
                
            }
            
            sorting.append(
            
                Product
            
            )
            
        return sorting
        

<br>

**[ * ] SORTING DATA**

In [6]:
class ecommerceData:
    
    def SORT(self, Search):
        
        toped = Tokopedia(Search).getTokopedia()
        shopee = Shopee(Search).getShopee()
        BL = Bukalapak(Search).getBukalapak()
        
        return pd.DataFrame(toped+shopee+BL).sort_values(by=["harga","rating","diskon"])


print("[ PLEASE WAIT...... ]\n")

ecommerceData().SORT(
    
    Search = "case xiaomi 4x" 

).to_csv("hasil.csv")

print("[ DONE...... ]\n")     

[ PLEASE WAIT...... ]

[ DONE...... ]

